## Initialization

In [ ]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
Note: the data was pickled with Python 2, and some encoding issues might prevent you
from loading it in Python 3. You might have to load it in Python 2,
save it in a different format, load it in Python 3 and repickle it.
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils

batch_size = 32
nb_classes = 10
nb_epoch = 1
data_augmentation = True
img_shape = (32, 32, 3)

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

In [ ]:
import pickle
import gzip

X_train, Y_train, X_test, Y_test = pickle.load(gzip.open("data.pkl.gz", 'rb')) # use normal pickle / cPickle for unpacking

## Model

In [ ]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=img_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

## Minibatch Generators (Pick one below)

In [ ]:
## Memory batch (Fast, of course)

def get_batches():
    batch_num = len(Y_train) // batch_size
    for i in range(batch_num):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_X = X_train[start: end]
        batch_Y = Y_train[start: end]
        batch_X = np.array(batch_X).astype('float32')
        batch_Y = np.array(batch_Y)
        yield batch_X, batch_Y

In [ ]:
## Directory batch (Very slow)
import glob
from scipy import misc

def get_batches():
    batch_num = len(Y_train) // batch_size
    for i in range(batch_num):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_X = [misc.imread("./images/%d.jpg" % j) for j in range(start, end)]
        batch_Y = Y_train[start: end]
        batch_X = np.array(batch_X).astype('float32') / 255
        batch_Y = np.array(batch_Y)
        yield batch_X, batch_Y

In [ ]:
## LMDB batch (Storage but fast)
import lmdb
import numpy as np

def get_batches():
    lmdb_env = lmdb.open('./data.lmdb')
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()
    
    batch_X, batch_Y = [], []
    for i, (k, x) in enumerate(lmdb_cursor):
        if len(batch_X) >= batch_size:
            batch_X = np.array(batch_X).astype('float32')
            batch_Y = np.array(batch_Y)
            yield batch_X, batch_Y
            batch_X, batch_Y = [], []
        else:
            batch_X.append(np.fromstring(x, dtype=np.float32).reshape(img_shape))
            batch_Y.append(Y_train[i])

## Training

In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz

from tqdm import tqdm
import numpy as np
num_batch_epoch = len(X_train) // batch_size
for epoch in range(nb_epoch):
    for x, y in tqdm(get_batches()):
        model.train_on_batch(x, y)
    print(model.evaluate(X_test, Y_test, batch_size=batch_size))

## Advanced techniques

More about profilers;

### Python profiler (CPU Time)

	python -m cProfile -o profile.log cifar10_cnn.py

### CUDA profiler

* edit .theanorc -> profile=True
* export CUDA_LAUNCH_BLOCKING=1

### Python profiler (Memory Usage)

	python -m memory_profiler cifar10_cnn.py
	


### GPU-related commands.

GPU Status;


```
$ nvidia-smi
$ nvidia-smi -q
$ watch nvidia-smi
```
	
Benchmark;

```
$ cp -Rf /usr/local/cuda/samples/ ~/
$ cd ~/samples/0_Simple/matrixMul ; make
$ ./matrixMul
```